This notebook works with the Taxonomic Information Registry. It picks up scientific names (currently only looking for those submitted by the SGCN process), queries the ITIS Solr service for matches, and caches a few specific properties in a key/value store.

In [15]:
import requests,configparser,re
from IPython.display import display

In [5]:
# Get API keys and any other config details from a file that is external to the code.
config = configparser.RawConfigParser()
config.read_file(open(r'../config/stuff.py'))

In [6]:
# Build base URL with API key using input from the external config.
def getBaseURL():
    gc2APIKey = config.get('apiKeys','apiKey_GC2_BCB').replace('"','')
    apiBaseURL = "https://gc2.mapcentia.com/api/v1/sql/bcb?key="+gc2APIKey
    return apiBaseURL

### Clean the scientific name string for use in searches
This is one of the more tricky areas of the process. People encode a lot of different signals into scientific names. If we clean too much out of the name string, we run the risk of not finding the taxon that they intended to provide. If we clean up too little, we won't find anything with the search. So far, for the SGCN case, we've decided to do the following in this code block for the purposes of finding the taxon in ITIS:

* Ignore population designations
* Ignore strings after an "spp." designation
* Set case for what appear to be species name strings to uppercase genus but lowercase everything else
* Ignore text in between parentheses and brackets; these are often synonyms or alternate names that should be picked up from the ITIS record if we find a match

One thing that I deliberately did not do here was change cases where the name string includes signals like "sp." or "sp. 4". Those are seeming to indicate that the genus is known but species is not yet determined. Rather than strip this text and run the query, potentially resulting in a genus match, I opted to leave those strings in place, likely resulting in no match with ITIS. We may end up making a different design decision for the SGCN case and allow for matching to genus. 

In [16]:
# There are a few things that we've found in the name strings that, if removed or modified, will result in a valid taxon name string for the ITIS service
def cleanScientificName(scientificname):
    # Get rid of "pop." from the string to enable the search to find a match without whatever population indicator is in the string
    if scientificname.find("pop."):
        nameParts = scientificname.split()
        regex = re.compile(r'pop\.[0-9]')
        scientificname = ' '.join([i for i in nameParts if not regex.search(i)])

    # Get rid of "spp." from the string, and it should find a genus match
    if scientificname.find("spp."):
        nameParts = scientificname.split()
        regex = re.compile(r'spp\.')
        scientificname = ' '.join([i for i in nameParts if not regex.search(i)])
    
    # Clean up all upper case strings because the ITIS service doesn't like them
    if any(x.isupper() for x in scientificname[-(len(scientificname)-1):]):
        scientificname = scientificname.lower().capitalize()
    
    # Get rid of text in parens and brackets; this is a design decision to potentially do away with information that might be important, but it gets retained in the original records
    scientificname = re.sub("[\(\[].*?[\)\]]", "", scientificname)
    scientificname = scientificname.replace("  "," ")

    return scientificname

In [23]:
def getITISSearchURL(scientificname):
    # Default to using name without indicator as the search term
    itisTerm = "nameWOInd"
    
    # "var." and "ssp." indicate that the string has population and variation indicators and should use the WInd service
    if scientificname.find("var.") > 0 or scientificname.find("ssp.") > 0:
        itisTerm = "nameWInd"
    
    # Put the search term together with the scientific name value including the weird escape character sequence that ITIS wants
    return "http://services.itis.gov/?wt=json&rows=10&q="+itisTerm+":"+scientificname.replace(" ","\%20")

### Package up the specific attributes we want to cache from ITIS
This function takes the data coming from the ITIS service as JSON and pairs up the attributes and values we want to cache and use. The date/time stamp here for when the information is cached is vital metadata for determining usability. As soon as the information comes out of ITIS, it is potentially stale. The information we collect and use from ITIS through this process includes the following:
* Discovered and accepted TSNs for the taxon
* Taxonomic rank of the discovered taxon
* Names with and without indicators for the discovered taxon
* Taxonomic hierarchy with ranks (in the ITIS Solr service, this is always the accepted taxonomic hierarchy)
* Vernacular names for the discovered taxon

In [44]:
def packageITISPairs(matchMethod,itisJSON):
    import datetime
    dt = datetime.datetime.utcnow().isoformat()
    itisPairs = '"cacheDate"=>"'+dt+'"'
    itisPairs = itisPairs+',"itisMatchMethod"=>"'+matchMethod+'"'

    if type(itisJSON) is int:
        return itisPairs
    else:
        itisPairs = itisPairs+',"createDate"=>"'+itisJSON['response']['docs'][0]['createDate']+'"'
        itisPairs = itisPairs+',"updateDate"=>"'+itisJSON['response']['docs'][0]['updateDate']+'"'
        itisPairs = itisPairs+',"discoveredTSN"=>"'+itisJSON['response']['docs'][0]['tsn']+'"'
        itisPairs = itisPairs+',"rank"=>"'+itisJSON['response']['docs'][0]['rank']+'"'
        itisPairs = itisPairs+',"nameWInd"=>"'+itisJSON['response']['docs'][0]['nameWInd']+'"'
        itisPairs = itisPairs+',"nameWOInd"=>"'+itisJSON['response']['docs'][0]['nameWOInd']+'"'

        if 'acceptedTSN' in itisJSON['response']['docs'][0]:
            itisPairs = itisPairs+',"acceptedTSN"=>"'+''.join(str(e) for e in itisJSON['response']['docs'][0]['acceptedTSN'])+'"'

        hierarchy = itisJSON['response']['docs'][0]['hierarchySoFarWRanks'][0]
        hierarchy = hierarchy[hierarchy.find(':$')+2:-1]
        hierarchy = '"'+hierarchy.replace(':', '"=>"').replace('$', '","')+'"'
        itisPairs = itisPairs+','+hierarchy

        if "vernacular" in itisJSON['response']['docs'][0]:
            vernacularList = []
            for commonName in itisJSON['response']['docs'][0]['vernacular']:
                commonNameElements = commonName.split('$')
                vernacularList.append('"vernacular:'+commonNameElements[2]+'"=>"'+commonNameElements[1]+'"')
            strVernacularList = ''.join(vernacularList).replace("\'", "''").replace('""','","')
            itisPairs = itisPairs+','+strVernacularList

        return itisPairs

In [45]:
# Basic function to insert subject ID, property, and value into tircache
def cacheToTIR(gid,infotype,pairs):
    import requests
    updateQ = "UPDATE tir.tir2 SET "+infotype+" = '"+pairs+"' WHERE gid = "+str(gid)
    r = requests.get(getBaseURL()+"&q="+updateQ).json()
    return r

In [46]:
# Query for the registered names we want to run through the system
uniqueNames  = requests.get(getBaseURL()+"&q=SELECT gid,registration->'SGCN_ScientificName_Submitted' AS scientificname FROM tir.tir2 WHERE itis IS NULL LIMIT 100").json()

### Run the process for all supplied names
This is the process that should eventually be the substance of a microservice on name matching. I set this up to create a local data structure (dictionary) for each record. The main point here is to set up the search, execute the search and package ITIS results, and then submit those for the record back to the Taxonomic Information Registry.

One of the major things we still need to work out here is what to do with updates over time. This script puts some record into the ITIS pairs whether we find a match or not. The query that gets names to run from the registration property looks for cases where the ITIS information is null (mostly because I needed to pick up where I left off when I found a few remaining issues that failed the script). We can then use cases where the "matchMethod" is "NotMatched" to go back and see if we can find name matches. This is particularly going to be the case where we find more than one match on a fuzzy search, which I still haven't dealt with.

We also need to figure out what to do when we want to update the information over time. With ITIS, once we have a matched TSN, we can then use that TSN to grab updates as they occur, including changes in taxonomy. But we need to figure out if we should change the structure of the TIR cache to keep all the old versions of what we found over time so that it can always be referred back to.

In [48]:
for feature in uniqueNames["features"]:
    # Set up a local data structure for storage and processing
    thisRecord = {}
    
    # Set data from query results
    thisRecord["gid"] = feature["properties"]["gid"]
    thisRecord["scientificname_submitted"] = feature["properties"]["scientificname"]
    thisRecord["scientificname_search"] = cleanScientificName(thisRecord["scientificname_submitted"])
    
    # Set defaults for thisRecord
    thisRecord["itisFoundFromFuzzy"] = 0
    thisRecord["matchMethod"] = "NotMatched:"+thisRecord["scientificname_search"]
    thisRecord["itisPairs"] = packageITISPairs(thisRecord["matchMethod"],0)

    # Handle the cases where there is enough interesting stuff in the scientific name string that it comes back blank from the cleaners
    if len(thisRecord["scientificname_search"]) != 0:
        # The ITIS Solr service does not fail in an elegant way, and so we need to try this whole section and except it out if the query fails
        try:
            thisRecord["itisSearchURL"] = getITISSearchURL(thisRecord["scientificname_search"])

            # Try an exact match search
            itisSearchResults = requests.get(thisRecord["itisSearchURL"]).json()
            thisRecord["itisFoundFromExact"] = itisSearchResults["response"]["numFound"]

            # Try a fuzzy match search and set match method
            if thisRecord["itisFoundFromExact"] == 0:
                itisSearchResults = requests.get(thisRecord["itisSearchURL"]+"~0.5").json()
                thisRecord["itisFoundFromFuzzy"] = itisSearchResults["response"]["numFound"]
                if thisRecord["itisFoundFromFuzzy"] == 1:
                    thisRecord["matchMethod"] = "FuzzyMatch:"+thisRecord["scientificname_search"]
            elif thisRecord["itisFoundFromExact"] == 1:
                thisRecord["matchMethod"] = "ExactMatch:"+thisRecord["scientificname_search"]

            # If there are results from exact or fuzzy match search, package the ITIS properties we want
            if itisSearchResults["response"]["numFound"] == 1:
                thisRecord["itisPairs"] = packageITISPairs(thisRecord["matchMethod"],itisSearchResults)
            else:
                thisRecord["itisPairs"] = packageITISPairs(thisRecord["matchMethod"],0)
        except:
            pass
    
    display (thisRecord)
    print (cacheToTIR(thisRecord["gid"],"itis",thisRecord["itisPairs"]))


{'gid': 21,
 'itisFoundFromExact': 1,
 'itisFoundFromFuzzy': 0,
 'itisPairs': '"cacheDate"=>"2017-05-06T15:54:05.316282","itisMatchMethod"=>"ExactMatch:Abagrotis brunneipennis","createDate"=>"2008-12-05 14:44:10","updateDate"=>"2014-05-28 00:00:00","discoveredTSN"=>"771341","rank"=>"Species","nameWInd"=>"Abagrotis brunneipennis","nameWOInd"=>"Abagrotis brunneipennis","Kingdom"=>"Animalia","Subkingdom"=>"Bilateria","Infrakingdom"=>"Protostomia","Superphylum"=>"Ecdysozoa","Phylum"=>"Arthropoda","Subphylum"=>"Hexapoda","Class"=>"Insecta","Subclass"=>"Pterygota","Infraclass"=>"Neoptera","Superorder"=>"Holometabola","Order"=>"Lepidoptera","Superfamily"=>"Noctuoidea","Family"=>"Noctuidae","Subfamily"=>"Noctuinae","Tribe"=>"Noctuini","Subtribe"=>"Noctuina","Genus"=>"Abagrotis","Species"=>"Abagrotis brunneipennis"',
 'itisSearchURL': 'http://services.itis.gov/?wt=json&rows=10&q=nameWOInd:Abagrotis\\%20brunneipennis',
 'matchMethod': 'ExactMatch:Abagrotis brunneipennis',
 'scientificname_search

{'success': True, 'auth_check': {'success': True, 'auth_level': None, 'session': None}, '_execution_time': 0.071, 'affected_rows': 1}


{'gid': 2286,
 'itisFoundFromExact': 1,
 'itisFoundFromFuzzy': 0,
 'itisPairs': '"cacheDate"=>"2017-05-06T15:54:06.132705","itisMatchMethod"=>"ExactMatch:Holospira sherbrookei","createDate"=>"2000-03-31 10:19:42","updateDate"=>"2001-10-17 00:00:00","discoveredTSN"=>"567703","rank"=>"Species","nameWInd"=>"Holospira sherbrookei","nameWOInd"=>"Holospira sherbrookei","Kingdom"=>"Animalia","Subkingdom"=>"Bilateria","Infrakingdom"=>"Protostomia","Superphylum"=>"Lophozoa","Phylum"=>"Mollusca","Class"=>"Gastropoda","Order"=>"Stylommatophora","Family"=>"Urocoptidae","Genus"=>"Holospira","Species"=>"Holospira sherbrookei","vernacular:English"=>"Silver Creek holospira"',
 'itisSearchURL': 'http://services.itis.gov/?wt=json&rows=10&q=nameWOInd:Holospira\\%20sherbrookei',
 'matchMethod': 'ExactMatch:Holospira sherbrookei',
 'scientificname_search': 'Holospira sherbrookei',
 'scientificname_submitted': 'Holospira sherbrookei'}

{'success': True, 'auth_check': {'success': True, 'auth_level': None, 'session': None}, '_execution_time': 0.068, 'affected_rows': 1}


{'gid': 11729,
 'itisFoundFromExact': 0,
 'itisFoundFromFuzzy': 1,
 'itisPairs': '"cacheDate"=>"2017-05-06T15:54:07.071556","itisMatchMethod"=>"FuzzyMatch:Lithobimorpha","createDate"=>"1998-05-14 13:57:58","updateDate"=>"2013-07-25 00:00:00","discoveredTSN"=>"553110","rank"=>"Order","nameWInd"=>"Lithobiomorpha","nameWOInd"=>"Lithobiomorpha","Kingdom"=>"Animalia","Subkingdom"=>"Bilateria","Infrakingdom"=>"Protostomia","Superphylum"=>"Ecdysozoa","Phylum"=>"Arthropoda","Subphylum"=>"Myriapoda","Class"=>"Chilopoda","Subclass"=>"Pleurostigmophora","Order"=>"Lithobiomorpha"',
 'itisSearchURL': 'http://services.itis.gov/?wt=json&rows=10&q=nameWOInd:Lithobimorpha',
 'matchMethod': 'FuzzyMatch:Lithobimorpha',
 'scientificname_search': 'Lithobimorpha',
 'scientificname_submitted': 'Lithobimorpha'}

...

{'success': True, 'auth_check': {'success': True, 'auth_level': None, 'session': None}, '_execution_time': 0.069, 'affected_rows': 1}


{'gid': 2342,
 'itisFoundFromExact': 1,
 'itisFoundFromFuzzy': 0,
 'itisPairs': '"cacheDate"=>"2017-05-06T15:55:41.411792","itisMatchMethod"=>"ExactMatch:Hymenophyllum tayloriae","createDate"=>"1996-11-04 17:57:08","updateDate"=>"2011-05-27 00:00:00","discoveredTSN"=>"507015","rank"=>"Species","nameWInd"=>"Hymenophyllum tayloriae","nameWOInd"=>"Hymenophyllum tayloriae","Kingdom"=>"Plantae","Subkingdom"=>"Viridiplantae","Infrakingdom"=>"Streptophyta","Superdivision"=>"Embryophyta","Division"=>"Tracheophyta","Subdivision"=>"Polypodiophytina","Class"=>"Polypodiopsida","Subclass"=>"Polypodiidae","Order"=>"Hymenophyllales","Family"=>"Hymenophyllaceae","Genus"=>"Hymenophyllum","Species"=>"Hymenophyllum tayloriae","vernacular:English"=>"Taylor\'\'s filmy fern"',
 'itisSearchURL': 'http://services.itis.gov/?wt=json&rows=10&q=nameWOInd:Hymenophyllum\\%20tayloriae',
 'matchMethod': 'ExactMatch:Hymenophyllum tayloriae',
 'scientificname_search': 'Hymenophyllum tayloriae',
 'scientificname_submitt

{'success': True, 'auth_check': {'success': True, 'auth_level': None, 'session': None}, '_execution_time': 0.069, 'affected_rows': 1}


{'gid': 111,
 'itisFoundFromExact': 1,
 'itisFoundFromFuzzy': 0,
 'itisPairs': '"cacheDate"=>"2017-05-06T15:55:42.396299","itisMatchMethod"=>"ExactMatch:Agkistrodon piscivorus","createDate"=>"1996-06-13 14:51:08","updateDate"=>"2005-07-29 00:00:00","discoveredTSN"=>"174299","rank"=>"Species","nameWInd"=>"Agkistrodon piscivorus","nameWOInd"=>"Agkistrodon piscivorus","Kingdom"=>"Animalia","Subkingdom"=>"Bilateria","Infrakingdom"=>"Deuterostomia","Phylum"=>"Chordata","Subphylum"=>"Vertebrata","Infraphylum"=>"Gnathostomata","Superclass"=>"Tetrapoda","Class"=>"Reptilia","Order"=>"Squamata","Suborder"=>"Serpentes","Infraorder"=>"Alethinophidia","Family"=>"Viperidae","Subfamily"=>"Crotalinae","Genus"=>"Agkistrodon","Species"=>"Agkistrodon piscivorus","vernacular:English"=>"Cottonmouth"',
 'itisSearchURL': 'http://services.itis.gov/?wt=json&rows=10&q=nameWOInd:Agkistrodon\\%20piscivorus',
 'matchMethod': 'ExactMatch:Agkistrodon piscivorus',
 'scientificname_search': 'Agkistrodon piscivorus',
 

{'success': True, 'auth_check': {'success': True, 'auth_level': None, 'session': None}, '_execution_time': 0.068, 'affected_rows': 1}
